# Clustering and Visualizing High Dimensional Data 

## Dataset

**Title:** Forest Cover Type Dataset 

**Description:** Tree types found in the Roosevelt National Forest in Colorado

**Context:** This dataset contains tree observations from four areas of the Roosevelt National Forest in Colorado. All observations are cartographic variables (no remote sensing) from 30 meter x 30 meter sections of forest. There are over half a million measurements total).

**Content:** This dataset includes information on tree type, shadow coverage, distance to nearby landmarks (roads etcetera), soil type, and local topography. 

**Acknowledgement:** This dataset is part of the UCI Machine Learning Repository, and the original source can be found here. The original database owners are Jock A. Blackard, Dr. Denis J. Dean, and Dr. Charles W. Anderson of the Remote Sensing and GIS Program at Colorado State University.

**URL:** https://www.kaggle.com/uciml/forest-cover-type-dataset

## Implementation

First we need to import all the necessary libraries

In [1]:
#Basic imports
import numpy as np
import pandas as pd

#sklearn imports
from sklearn.decomposition import PCA #Principal Component Analysis
from sklearn.manifold import TSNE #T-Distributed Stochastic Neighbor Embedding
from sklearn.cluster import KMeans #K-Means Clustering
from sklearn.preprocessing import StandardScaler #used for 'Feature Scaling'

#plotly imports
import plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

Now, let's import the dataset

In [2]:
df = pd.read_csv("data/covtype.csv")

Before proceeding, we have to make a clone of the dataframe. Then we need to check whether if there are any missing values as follows.

#### Preprocessing

In [3]:
X = df.copy()
X.isnull().sum()

Elevation                             0
Aspect                                0
Slope                                 0
Horizontal_Distance_To_Hydrology      0
Vertical_Distance_To_Hydrology        0
Horizontal_Distance_To_Roadways       0
Hillshade_9am                         0
Hillshade_Noon                        0
Hillshade_3pm                         0
Horizontal_Distance_To_Fire_Points    0
Wilderness_Area1                      0
Wilderness_Area2                      0
Wilderness_Area3                      0
Wilderness_Area4                      0
Soil_Type1                            0
Soil_Type2                            0
Soil_Type3                            0
Soil_Type4                            0
Soil_Type5                            0
Soil_Type6                            0
Soil_Type7                            0
Soil_Type8                            0
Soil_Type9                            0
Soil_Type10                           0
Soil_Type11                           0


Since there are no missing values, we can proceed examine the dataset a bit.

There are 581,012 entries and 55 attributes. Among them, 10 are numerical and 45 is categorical. The categorical variable 'Cover_Type' is currently assigned with integers which represents the following cover types according to https://archive.ics.uci.edu/ml/datasets/Covertype
    1. Spruce/Fir
    2. Lodgepole Pine
    3. Ponderosa Pine
    4. Cottonwood/Willow
    5. Aspen
    6. Douglas-fir
    7. Krummholz

In [4]:
X.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
0,2596,51,3,258,0,510,221,232,148,6279,...,0,0,0,0,0,0,0,0,0,5
1,2590,56,2,212,-6,390,220,235,151,6225,...,0,0,0,0,0,0,0,0,0,5
2,2804,139,9,268,65,3180,234,238,135,6121,...,0,0,0,0,0,0,0,0,0,2
3,2785,155,18,242,118,3090,238,238,122,6211,...,0,0,0,0,0,0,0,0,0,2
4,2595,45,2,153,-1,391,220,234,150,6172,...,0,0,0,0,0,0,0,0,0,5


According to the following data description, there are 54 attributes consititute to make a decision on 'Cover_Type'. So, if we were to do clustering this is will be considered as a high dimensional set of data as the no.of dimensions are greater than 3. 

To calculate the distance between two points within a spacial space of 54 dimensions is possible but difficult for the algorithms.

Also, some of the dimensions have values in ranges varying from tens to hundreds to thousands. So, distance calculations using these values will only hinder the performance of the clustering algorithm.

Therefore, we need to normalize the data attributes and then apply dimensionality reduction techniques before going in for clustering.

In [5]:
X[["Elevation","Aspect","Slope","Horizontal_Distance_To_Roadways","Hillshade_9am","Hillshade_Noon","Hillshade_3pm","Horizontal_Distance_To_Fire_Points","Horizontal_Distance_To_Hydrology","Vertical_Distance_To_Hydrology"]].describe()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology
count,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000
mean,2959.365301,155.656807,14.103704,2350.146611,212.146049,223.318716,142.528263,1980.291226,269.428217,46.418855
std,279.984734,111.913721,7.488242,1559.254870,26.769889,19.768697,38.274529,1324.195210,212.549356,58.295232
min,1859.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-173.000000
25%,2809.000000,58.000000,9.000000,1106.000000,198.000000,213.000000,119.000000,1024.000000,108.000000,7.000000
50%,2996.000000,127.000000,13.000000,1997.000000,218.000000,226.000000,143.000000,1710.000000,218.000000,30.000000
75%,3163.000000,260.000000,18.000000,3328.000000,231.000000,237.000000,168.000000,2550.000000,384.000000,69.000000
max,3858.000000,360.000000,66.000000,7117.000000,254.000000,254.000000,254.000000,7173.000000,1397.000000,601.000000


When we observe the numerical attribute, we can see that we don't need the following two separate variables.
  - Horizontal_Distance_To_Hydrology
  - Vertical_Distance_To_Hydrology

We can combine them into one attribute and remove the above two as follows.

In [6]:
X["Distance_To_Hydrology"] = ( (X["Horizontal_Distance_To_Hydrology"] ** 2) + (X["Vertical_Distance_To_Hydrology"] ** 2) ) ** (0.5)

In [7]:
X.drop(["Horizontal_Distance_To_Hydrology","Vertical_Distance_To_Hydrology"], axis=1, inplace=True)

In order to one-hot-encode the 'Cover_Type' attribute, let's add its corresponding class labels as follows and then do the one-hot-encoding.

In [8]:
X['Cover_Type'].replace({1:'Spruce/Fir', 2:'Lodgepole Pine', 3:'Ponderosa Pine', 4:'Cottonwood/Willow', 5:'Aspen', 6:'Douglas-fir', 7:'Krummholz'}, inplace=True)

In [9]:
X = pd.get_dummies(X, columns=['Cover_Type'])

,Elevation,Aspect,Slope,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area1,Wilderness_Area2,...,Soil_Type39,Soil_Type40,Distance_To_Hydrology,Cover_Type_Aspen,Cover_Type_Cottonwood/Willow,Cover_Type_Douglas-fir,Cover_Type_Krummholz,Cover_Type_Lodgepole Pine,Cover_Type_Ponderosa Pine,Cover_Type_Spruce/Fir
0,2596,51,3,510,221,232,148,6279,1,0,...,0,0,258.000000,1,0,0,0,0,0,0
1,2590,56,2,390,220,235,151,6225,1,0,...,0,0,212.084889,1,0,0,0,0,0,0
2,2804,139,9,3180,234,238,135,6121,1,0,...,0,0,275.769832,0,0,0,0,1,0,0
3,2785,155,18,3090,238,238,122,6211,1,0,...,0,0,269.235956,0,0,0,0,1,0,0
4,2595,45,2,391,220,234,150,6172,1,0,...,0,0,153.003268,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581007,2396,153,20,108,240,237,118,837,0,0,...,0,0,86.683332,0,0,0,0,0,1,0
581008,2391,152,19,95,240,237,119,845,0,0,...,0,0,68.066144,0,0,0,0,0,1,0
581009,2386,159,17,90,236,241,130,854,0,0,...,0,0,60.406953,0,0,0,0,0,1,0
581010,2384,170,15,90,230,245,143,864,0,0,...,0,0,60.207973,0,0,0,0,0,1,0


#### Normalization

Let's divide the data set into numerical and categorical attributes before doing the normalization as follows.

In [11]:
numerical = X[["Elevation","Aspect","Slope","Horizontal_Distance_To_Roadways","Hillshade_9am","Hillshade_Noon","Hillshade_3pm","Horizontal_Distance_To_Fire_Points","Distance_To_Hydrology"]]
categorical = X[["Wilderness_Area1","Wilderness_Area2","Wilderness_Area3","Wilderness_Area4","Soil_Type1","Soil_Type2","Soil_Type3","Soil_Type4","Soil_Type5","Soil_Type6","Soil_Type7","Soil_Type8","Soil_Type9","Soil_Type10","Soil_Type11","Soil_Type12","Soil_Type13","Soil_Type14","Soil_Type15","Soil_Type16","Soil_Type17","Soil_Type18","Soil_Type19","Soil_Type20","Soil_Type21","Soil_Type22","Soil_Type23","Soil_Type24","Soil_Type25","Soil_Type26","Soil_Type27","Soil_Type28","Soil_Type29","Soil_Type30","Soil_Type31","Soil_Type32","Soil_Type33","Soil_Type34","Soil_Type35","Soil_Type36","Soil_Type37","Soil_Type38","Soil_Type39","Soil_Type40","Cover_Type_Aspen","Cover_Type_Cottonwood/Willow","Cover_Type_Douglas-fir","Cover_Type_Krummholz","Cover_Type_Lodgepole Pine","Cover_Type_Ponderosa Pine","Cover_Type_Spruce/Fir"]]

Now we can do the normalization.

In [15]:
sc = StandardScaler()
numerical = pd.DataFrame(sc.fit_transform(numerical))
numerical.columns = ["Elevation_Scaled","Aspect_Scaled","Slope_Scaled","Horizontal_Distance_To_Roadways_Scaled","Hillshade_9am_Scaled","Hillshade_Noon_Scaled","Hillshade_3pm_Scaled","Horizontal_Distance_To_Fire_Points_Scaled","Distance_To_Hydrology_Scaled"]

Now we can rejoin the two variable sets together to make the full dataset.

In [16]:
X = pd.concat([numerical, categorical], axis=1, join='inner')
X.head()

,Elevation_Scaled,Aspect_Scaled,Slope_Scaled,Horizontal_Distance_To_Roadways_Scaled,Hillshade_9am_Scaled,Hillshade_Noon_Scaled,Hillshade_3pm_Scaled,Horizontal_Distance_To_Fire_Points_Scaled,Distance_To_Hydrology_Scaled,Wilderness_Area1,...,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type_Aspen,Cover_Type_Cottonwood/Willow,Cover_Type_Douglas-fir,Cover_Type_Krummholz,Cover_Type_Lodgepole Pine,Cover_Type_Ponderosa Pine,Cover_Type_Spruce/Fir
0,-1.297805,-0.935157,-1.482820,-1.180146,0.330743,0.439143,0.142960,3.246283,-0.083233,1,...,0,0,0,1,0,0,0,0,0,0
1,-1.319235,-0.890480,-1.616363,-1.257106,0.293388,0.590899,0.221342,3.205504,-0.294777,1,...,0,0,0,1,0,0,0,0,0,0
2,-0.554907,-0.148836,-0.681563,0.532212,0.816364,0.742654,-0.196691,3.126965,-0.001362,1,...,0,0,0,0,0,0,0,1,0,0
3,-0.622768,-0.005869,0.520322,0.474492,0.965786,0.742654,-0.536343,3.194931,-0.031466,1,...,0,0,0,0,0,0,0,1,0,0
4,-1.301377,-0.988770,-1.616363,-1.256464,0.293388,0.540313,0.195215,3.165479,-0.566983,1,...,0,0,0,1,0,0,0,0,0,0


In [22]:
pca_1d = PCA(n_components=1)

pca_2d = PCA(n_components=2)

pca_3d = PCA(n_components=3)

In [41]:
PCs_1d = pd.DataFrame(pca_1d.fit_transform(X))

PCs_2d = pd.DataFrame(pca_2d.fit_transform(X))

PCs_3d = pd.DataFrame(pca_3d.fit_transform(X))

In [42]:
PCs_1d.columns = ["PC1_1d"]
PCs_2d.columns = ["PC1_2d", "PC2_2d"]
PCs_3d.columns = ["PC1_3d", "PC2_3d", "PC3_3d"]

In [59]:
kmeans_1 = KMeans(n_clusters=3)
kmeans_2 = KMeans(n_clusters=3)
kmeans_3 = KMeans(n_clusters=3)

In [60]:
kmeans_1.fit(PCs_1d)

KMeans(n_clusters=3)

In [61]:
kmeans_2.fit(PCs_2d)

KMeans(n_clusters=3)

In [62]:
kmeans_3.fit(PCs_3d)

KMeans(n_clusters=3)

In [63]:
clusters_1 = kmeans_1.predict(PCs_1d)

In [64]:
clusters_2 = kmeans_2.predict(PCs_2d)

In [65]:
clusters_3 = kmeans_3.predict(PCs_3d)

In [66]:
PCs_1d["Cluster"] = clusters_1
PCs_2d["Cluster"] = clusters_2
PCs_3d["Cluster"] = clusters_3

In [69]:
PCs_1d["Cluster"]

2    224313
0    202791
1    153908
Name: Cluster, dtype: int64

In [ ]:
#Instructions for building the 2-D plot

#trace1 is for 'Cluster 0'
trace1 = go.Scatter(
                    x = cluster0["PC1_2d"],
                    y = cluster0["PC2_2d"],
                    mode = "markers",
                    name = "Cluster 0",
                    marker = dict(color = 'rgba(255, 128, 255, 0.8)'),
                    text = None)

#trace2 is for 'Cluster 1'
trace2 = go.Scatter(
                    x = cluster1["PC1_2d"],
                    y = cluster1["PC2_2d"],
                    mode = "markers",
                    name = "Cluster 1",
                    marker = dict(color = 'rgba(255, 128, 2, 0.8)'),
                    text = None)

#trace3 is for 'Cluster 2'
trace3 = go.Scatter(
                    x = cluster2["PC1_2d"],
                    y = cluster2["PC2_2d"],
                    mode = "markers",
                    name = "Cluster 2",
                    marker = dict(color = 'rgba(0, 255, 200, 0.8)'),
                    text = None)

data = [trace1, trace2, trace3]

title = "Visualizing Clusters in Two Dimensions Using PCA"

layout = dict(title = title,
              xaxis= dict(title= 'PC1',ticklen= 5,zeroline= False),
              yaxis= dict(title= 'PC2',ticklen= 5,zeroline= False)
             )

fig = dict(data = data, layout = layout)

iplot(fig)